# Регрессия

In [ ]:
#! pip uninstall stargazer
!pip install statsmodels
! pip install git+https://github.com/StatsReporting/stargazer
#! pip install stargazer

from stargazer.stargazer import Stargazer

  Cloning https://github.com/StatsReporting/stargazer to /tmp/pip-req-build-46gj66ob
  Running command git clone --filter=blob:none --quiet https://github.com/StatsReporting/stargazer /tmp/pip-req-build-46gj66ob
  Resolved https://github.com/StatsReporting/stargazer to commit 007d33f5d0cba473f2c9529b9a5698f3648b117c
  Preparing metadata (setup.py) ... done


In [ ]:
! pip install openpyxl

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
fin_df_raw = pd.read_csv('fin_df.csv')
fin_q_df_raw = pd.read_excel('fin_quarter.xlsx')
df_raw = pd.read_excel('classified.xlsx', engine='openpyxl')

## Merge

### Финансовые данные (ежедневные)

In [ ]:
fin_df = fin_df_raw[['Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividends', 'Stock Splits', 'Diluted Average Shares', 'Market_Cap',
       'Ticker', 'company_name', 'industry', 'sector', 'revenue_growth',
       'SP500_returns', 'SP500_returns_yesterday', 'Returns', 'Returns_binary',
       'Returns_yesterday']]

fin_df['Date'] = pd.to_datetime(fin_df['Date'], errors='coerce')
fin_df.head()

<ipython-input-5-331cb1a86a81>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df['Date'] = pd.to_datetime(fin_df['Date'], errors='coerce')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Diluted Average Shares,Market_Cap,Ticker,company_name,industry,sector,revenue_growth,SP500_returns,SP500_returns_yesterday,Returns,Returns_binary,Returns_yesterday
0,2019-01-02,36.944466,37.889009,36.787041,37.667183,148158800,0.0,0.0,1.686492e+10,6.352540e+11,AAPL,Apple Inc.,Consumer Electronics,Technology,0.04,0.013351,-0.017669,0.019562,1,-0.012432
1,2019-01-03,34.342195,34.757222,33.869926,33.915245,365248800,0.0,0.0,1.686492e+10,5.719779e+11,AAPL,Apple Inc.,Consumer Electronics,Technology,0.04,-0.017669,0.023283,-0.012432,0,0.025808
2,2019-01-04,34.473394,35.432248,34.299275,35.363075,234428400,0.0,0.0,1.686492e+10,5.963954e+11,AAPL,Apple Inc.,Consumer Electronics,Technology,0.04,0.023283,0.005553,0.025808,1,-0.005178
3,2019-01-07,35.468025,35.499034,34.800166,35.284363,219111200,0.0,0.0,1.686492e+10,5.950679e+11,AAPL,Apple Inc.,Consumer Electronics,Technology,0.04,0.005553,0.002453,-0.005178,0,0.007957
4,2019-01-08,35.673145,36.212204,35.425085,35.956985,164101200,0.0,0.0,1.686492e+10,6.064116e+11,AAPL,Apple Inc.,Consumer Electronics,Technology,0.04,0.002453,0.001922,0.007957,1,0.013352


In [ ]:
def compute_beta(group):
    if len(group) < 2:
        return np.nan
    cov = np.cov(group['Returns'], group['SP500_returns'])[0][1]
    var = np.var(group['SP500_returns'])
    beta = cov / var if var != 0 else np.nan
    return beta

betas = fin_df.groupby('Ticker').apply(compute_beta).reset_index()
betas.columns = ['Ticker', 'Beta']
fin_df = fin_df.merge(betas, on='Ticker', how='left')

fin_df['Abnormal_Return'] = fin_df['Returns'] - (fin_df['SP500_returns'] * fin_df['Beta'])

<ipython-input-6-5df0a1ec3c7e>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  betas = fin_df.groupby('Ticker').apply(compute_beta).reset_index()


### ФИнансовые данные ежеквартальные

In [ ]:
fin_q_df = fin_q_df_raw.copy(deep=True)
fin_q_df

,Date,Ticker,Total Assets,ROA,ROE,LEV,P/E,PBV,EPS
0,2024-12-31,AAPL,"$344,09B",0.2791,145.35%,4.15,39.71,56.36,$2.40
1,2024-09-30,AAPL,"$364,98B",0.2702,137.87%,5.41,38.24,61.71,$0.97
2,2024-06-30,AAPL,"$331,61B",0.2966,147.15%,3.97,31.95,47.90,$1.40
3,2024-03-31,AAPL,"$337,41B",0.2913,148.33%,3.55,26.54,35.28,$1.53
4,2023-12-31,AAPL,"$353,51B",0.2939,156.04%,3.77,29.81,39.93,$2.18
...,...,...,...,...,...,...,...,...,...
1719,2019-10-31,DOCU,$1.79B,-13.22%,-38.84%,2.14,0,20.84,$-0.26
1720,2019-07-31,DOCU,$1.76B,-13.70%,-36.15%,2.11,0,16.06,$-0.39
1721,2019-04-30,DOCU,$1.74B,-12.94%,-29.22%,1.91,0,16.46,$-0.27
1722,2019-01-31,DOCU,$1.62B,-33.37%,-71.73%,1.63,0,13.63,$4.83


In [ ]:
# Обработка
def parse_dollar_value(s):
    if pd.isna(s):
        return None
    s = str(s).replace('$', '').replace(',', '.')
    if s.endswith('B'):
        return float(s[:-1]) * 1e9
    elif s.endswith('M'):
        return float(s[:-1]) * 1e6
    else:
        return float(s)

# Функция для обработки ROA
def parse_roa(value):
    try:
        return float(value)
    except ValueError:
        return float(value.replace('%', '').replace(',', '.')) / 100

fin_q_df['Total Assets'] = fin_q_df['Total Assets'].apply(parse_dollar_value)
fin_q_df['EPS'] = fin_q_df['EPS'].apply(lambda x: parse_dollar_value(x))
fin_q_df['ROE'] = fin_q_df['ROE'].str.replace('%', '').astype(float) / 100
fin_q_df['ROA'] = fin_q_df['ROA'].apply(parse_roa)
fin_q_df['P/E'] = pd.to_numeric(fin_q_df['P/E'], errors='coerce')
fin_q_df['Date'] = pd.to_datetime(fin_q_df['Date'])
fin_q_df

,Date,Ticker,Total Assets,ROA,ROE,LEV,P/E,PBV,EPS
0,2024-12-31,AAPL,3.440900e+11,0.2791,1.4535,4.15,39.71,56.36,2.40
1,2024-09-30,AAPL,3.649800e+11,0.2702,1.3787,5.41,38.24,61.71,0.97
2,2024-06-30,AAPL,3.316100e+11,0.2966,1.4715,3.97,31.95,47.90,1.40
3,2024-03-31,AAPL,3.374100e+11,0.2913,1.4833,3.55,26.54,35.28,1.53
4,2023-12-31,AAPL,3.535100e+11,0.2939,1.5604,3.77,29.81,39.93,2.18
...,...,...,...,...,...,...,...,...,...
1719,2019-10-31,DOCU,1.790000e+09,-0.1322,-0.3884,2.14,0.00,20.84,-0.26
1720,2019-07-31,DOCU,1.760000e+09,-0.1370,-0.3615,2.11,0.00,16.06,-0.39
1721,2019-04-30,DOCU,1.740000e+09,-0.1294,-0.2922,1.91,0.00,16.46,-0.27
1722,2019-01-31,DOCU,1.620000e+09,-0.3337,-0.7173,1.63,0.00,13.63,4.83


### Объединение ежедневных и квартальных

In [ ]:
fin_df = fin_df.copy()
fin_q_df = fin_q_df.copy()

fin_df['Date'] = pd.to_datetime(fin_df['Date'])
fin_q_df['Date'] = pd.to_datetime(fin_q_df['Date'])

fin_df['YearQuarter'] = fin_df['Date'].dt.to_period('Q')
fin_q_df['YearQuarter'] = fin_q_df['Date'].dt.to_period('Q')


merged_df = pd.merge(fin_df, fin_q_df.drop(columns='Date'), on=['Ticker', 'YearQuarter'], how='left')
merged_df.drop(columns=['YearQuarter'], inplace=True)
merged_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Diluted Average Shares,Market_Cap,...,Returns_yesterday,Beta,Abnormal_Return,Total Assets,ROA,ROE,LEV,P/E,PBV,EPS
0,2019-01-02,36.944466,37.889009,36.787041,37.667183,148158800,0.0,0.0,1.686492e+10,6.352540e+11,...,-0.012432,1.120025,0.004609,3.420000e+11,0.1598,0.5129,2.23,15.31,13.96,0.62
1,2019-01-03,34.342195,34.757222,33.869926,33.915245,365248800,0.0,0.0,1.686492e+10,5.719779e+11,...,0.025808,1.120025,0.007358,3.420000e+11,0.1598,0.5129,2.23,15.31,13.96,0.62
2,2019-01-04,34.473394,35.432248,34.299275,35.363075,234428400,0.0,0.0,1.686492e+10,5.963954e+11,...,-0.005178,1.120025,-0.000270,3.420000e+11,0.1598,0.5129,2.23,15.31,13.96,0.62
3,2019-01-07,35.468025,35.499034,34.800166,35.284363,219111200,0.0,0.0,1.686492e+10,5.950679e+11,...,0.007957,1.120025,-0.011398,3.420000e+11,0.1598,0.5129,2.23,15.31,13.96,0.62
4,2019-01-08,35.673145,36.212204,35.425085,35.956985,164101200,0.0,0.0,1.686492e+10,6.064116e+11,...,0.013352,1.120025,0.005209,3.420000e+11,0.1598,0.5129,2.23,15.31,13.96,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106883,2024-12-24,95.330002,96.320000,94.591003,94.849998,1287900,0.0,0.0,2.089500e+08,1.981891e+10,...,0.018396,1.293798,-0.017014,3.770000e+09,0.3013,0.6505,0.90,14.31,7.04,0.30
106884,2024-12-26,94.040001,95.870003,93.949997,95.769997,1518700,0.0,0.0,2.089500e+08,2.001114e+10,...,-0.024134,1.293798,0.015686,3.770000e+09,0.3013,0.6505,0.90,14.31,7.04,0.30
106885,2024-12-27,95.300003,95.449997,92.169998,93.000000,1510700,0.0,0.0,2.089500e+08,1.943235e+10,...,-0.004343,1.293798,-0.016524,3.770000e+09,0.3013,0.6505,0.90,14.31,7.04,0.30
106886,2024-12-30,92.099998,93.000000,90.639999,91.699997,2442900,0.0,0.0,2.089500e+08,1.916071e+10,...,-0.017049,1.293798,-0.001343,3.770000e+09,0.3013,0.6505,0.90,14.31,7.04,0.30


Заполняю пропуски

In [ ]:
merged_df['Year'] = merged_df['Date'].dt.year
numeric_cols = merged_df.select_dtypes(include='number').columns
mean_values = merged_df.groupby(['Ticker', 'Year'])[numeric_cols].transform('mean')
merged_df[numeric_cols] = merged_df[numeric_cols].fillna(mean_values)
fin_df = merged_df.set_index(['Ticker', 'Date'])

In [ ]:
# Группируем по тикерам и считаем количество пропусков в каждом столбце
ticker_nulls = fin_df.groupby('Ticker').apply(lambda x: x.isnull().sum())

# Отобразим статистику по пропускам для каждого тикера
ticker_nulls.loc[:, 'Returns_yesterday':][ticker_nulls.loc[:, 'Returns_yesterday':].sum(axis=1) > 0]

,Returns_yesterday,Beta,Abnormal_Return,Total Assets,ROA,ROE,LEV,P/E,PBV,EPS,Year
Ticker,,,,,,,,,,,
AI,0,0,0,0,0,0,0,16,16,0,0
BBAI,0,0,0,0,0,0,0,190,0,0,0
BLUE,0,0,0,505,505,505,505,505,505,505,0
BTBT,0,0,0,252,252,252,0,252,0,0,0
BYND,0,0,0,0,0,0,0,169,0,0,0
CHPT,0,0,0,75,75,75,75,1523,75,75,0
CHWY,0,0,0,0,0,0,0,139,0,0,0
COIN,0,0,0,0,0,0,0,183,0,0,0
CRSR,0,0,0,0,0,0,0,70,0,0,0


In [ ]:
fin_df = fin_df.reset_index()
fin_df = fin_df[(fin_df['Ticker'] != 'NKLA') & (fin_df['Ticker'] != 'ON') & (fin_df['Ticker'] != 'BLUE')]

In [ ]:
numerical_columns = fin_df.select_dtypes(include='number').columns
mean_values_per_ticker = fin_df.groupby('Ticker')[numerical_columns].transform('mean')  # Средние значения по тикеру

# Заполняю пропуски средними значениями по тикеру
fin_df[numerical_columns] = fin_df[numerical_columns].fillna(mean_values_per_ticker)
fin_df = fin_df.fillna(0) # оставшиеся нулями
fin_df = fin_df.set_index(['Ticker', 'Date'])
print(fin_df.isnull().sum())

Open                       0
High                       0
Low                        0
Close                      0
Volume                     0
Dividends                  0
Stock Splits               0
Diluted Average Shares     0
Market_Cap                 0
company_name               0
industry                   0
sector                     0
revenue_growth             0
SP500_returns              0
SP500_returns_yesterday    0
Returns                    0
Returns_binary             0
Returns_yesterday          0
Beta                       0
Abnormal_Return            0
Total Assets               0
ROA                        0
ROE                        0
LEV                        0
P/E                        0
PBV                        0
EPS                        0
Year                       0
dtype: int64


### News df

In [ ]:
df = df_raw.copy(deep=True)
#df = df[df['Category'] != 'Other']

df['Date'] = df['Date'].replace('-', pd.NaT)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna()
df = df.set_index('Date')
df#.head()

,Title,Content,Publisher,Ticker,Type,Year,sentiment_label,sentiment_score,fakenews_label,fakenews_score,Category
Date,,,,,,,,,,,
2025-03-05 09:00:00,Apple introduces the new MacBook Air with the ...,"CUPERTINO, Calif.--(BUSINESS WIRE)--Apple® tod...",official press release,AAPL,press release,2025,positive,0.446794,LABEL_0,0.999664,Application
2025-03-05 09:00:00,"Apple unveils new Mac Studio, the most powerfu...","CUPERTINO, Calif.--(BUSINESS WIRE)--Apple® tod...",official press release,AAPL,press release,2025,neutral,0.644222,LABEL_0,0.993358,Other
2025-03-05 09:00:00,"Apple reveals M3 Ultra, taking Apple silicon t...","CUPERTINO, Calif.--(BUSINESS WIRE)--Apple® tod...",official press release,AAPL,press release,2025,neutral,0.700092,LABEL_1,0.942125,Other
2025-03-04 09:00:00,Apple introduces iPad Air with powerful M3 chi...,"CUPERTINO, Calif.--(BUSINESS WIRE)--Apple® tod...",official press release,AAPL,press release,2025,positive,0.518825,LABEL_0,0.969749,Application
2025-02-24 06:00:00,Apple will spend more than $500 billion in the...,"CUPERTINO, Calif.--(BUSINESS WIRE)--Apple® tod...",official press release,AAPL,press release,2025,neutral,0.739777,LABEL_1,0.927906,Other
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30 08:00:00,Winners And Losers Of Q3: Zoom (NASDAQ:ZM) Vs ...,Winners And Losers Of Q3: Zoom (NASDAQ:ZM) Vs ...,Yahoo Finance,ZM,news,2024,neutral,0.8978,LABEL_0,0.997091,Other
2024-12-30 08:00:00,Video conferencing giant Zoom to set up first ...,Video conferencing giant Zoom to set up first ...,The Business Journals,ZM,news,2024,neutral,0.808807,LABEL_0,0.988997,Other
2024-12-30 08:00:00,Zoom Video Poised for Growth in 2025: Time to ...,Zoom Video Poised for Growth in 2025: Time to ...,Yahoo Finance,ZM,news,2024,neutral,0.897468,LABEL_0,0.958526,Other


In [ ]:
df['Category'] = df['Category'].replace({'Artificial Intelligence': 'AI', 'Big Data': 'BigData', 'Cloud Computing': 'Cloud',
                                         'Block Chain': 'BlockChain', 'Application ': 'Application'})

In [ ]:
news_clean = df[['sentiment_label', 'fakenews_label', 'Ticker', 'Type', 'Category']]
news_clean_encoded = pd.get_dummies(news_clean, columns=['sentiment_label', 'fakenews_label', "Type", 'Category'], prefix=['sentiment', 'fakenews', 'type', 'сategory'])

print(news_clean_encoded.columns)
news_clean_encoded[['sentiment_negative', 'sentiment_neutral','sentiment_positive', 'fakenews_LABEL_0',
                    'fakenews_LABEL_1','type_news', 'type_press release', 'сategory_AI', 'сategory_Application',
                    'сategory_BigData', 'сategory_BlockChain', 'сategory_Cloud']] = news_clean_encoded[['sentiment_negative',
                                                                                                 'sentiment_neutral',
                                                                                                 'sentiment_positive',
                                                                                                 'fakenews_LABEL_0',
                                                                                                 'fakenews_LABEL_1',
                                                                                                 'type_news',
                                                                                                 'type_press release',
                                                                                                'сategory_AI', 'сategory_Application', 'сategory_BigData', 'сategory_BlockChain', 'сategory_Cloud']].astype(int)


##news_clean_encoded_daily = news_clean_encoded.resample('D').agg(agg_func)
news_clean_encoded['Pos_news'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news']
news_clean_encoded['Neg_news'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news']
news_clean_encoded['Neut_news'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news']
news_clean_encoded['Pos_PressRelease'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release']
news_clean_encoded['Neg_PressRelease'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release']
news_clean_encoded['Neut_PressRelease'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release']

# 'сategory_AI',
news_clean_encoded['Pos_news_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_news_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_news_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Pos_PressRelease_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_PressRelease_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_PressRelease_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']

# 'сategory_Application',
news_clean_encoded['Pos_news_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_news_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_news_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Pos_PressRelease_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_PressRelease_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_PressRelease_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']

# 'сategory_BigData',
news_clean_encoded['Pos_news_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_news_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_news_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Pos_PressRelease_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_PressRelease_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_PressRelease_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']

#  'сategory_BlockChain',
news_clean_encoded['Pos_news_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_news_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_news_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Pos_PressRelease_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_PressRelease_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_PressRelease_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']

# 'сategory_Cloud'
news_clean_encoded['Pos_news_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_news_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_news_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Pos_PressRelease_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_PressRelease_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_PressRelease_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']


########################################################################
news_clean_encoded['Pos_Tr_news'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news']
news_clean_encoded['Neut_Tr_news'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news']
news_clean_encoded['Neg_Tr_news'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news']
news_clean_encoded['Pos_Fake_news'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news']
news_clean_encoded['Neut_Fake_news'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news']
news_clean_encoded['Neg_Fake_news'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news']

news_clean_encoded['Pos_Tr_PressRelease'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release']
news_clean_encoded['Neut_Tr_PressRelease'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release']
news_clean_encoded['Neg_Tr_PressRelease'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release']
news_clean_encoded['Pos_Fake_PressRelease'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release']
news_clean_encoded['Neut_Fake_PressRelease'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release']
news_clean_encoded['Neg_Fake_PressRelease'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release']


# 'сategory_AI',
########################################################################
news_clean_encoded['Pos_Tr_news_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_Tr_news_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_Tr_news_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Pos_Fake_news_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_Fake_news_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_Fake_news_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_AI']

news_clean_encoded['Pos_Tr_PressRelease_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_Tr_PressRelease_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_Tr_PressRelease_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Pos_Fake_PressRelease_AI'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neut_Fake_PressRelease_AI'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']
news_clean_encoded['Neg_Fake_PressRelease_AI'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_AI']

# 'сategory_Application',
########################################################################
news_clean_encoded['Pos_Tr_news_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_Tr_news_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_Tr_news_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Pos_Fake_news_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_Fake_news_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_Fake_news_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Application']

news_clean_encoded['Pos_Tr_PressRelease_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_Tr_PressRelease_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_Tr_PressRelease_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Pos_Fake_PressRelease_Application'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neut_Fake_PressRelease_Application'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']
news_clean_encoded['Neg_Fake_PressRelease_Application'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Application']

# 'сategory_BigData',
########################################################################
news_clean_encoded['Pos_Tr_news_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_Tr_news_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_Tr_news_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Pos_Fake_news_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_Fake_news_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_Fake_news_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BigData']

news_clean_encoded['Pos_Tr_PressRelease_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_Tr_PressRelease_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_Tr_PressRelease_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Pos_Fake_PressRelease_BigData'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neut_Fake_PressRelease_BigData'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']
news_clean_encoded['Neg_Fake_PressRelease_BigData'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BigData']

#  'сategory_BlockChain',
########################################################################
news_clean_encoded['Pos_Tr_news_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_Tr_news_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_Tr_news_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Pos_Fake_news_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_Fake_news_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_Fake_news_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']

news_clean_encoded['Pos_Tr_PressRelease_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_Tr_PressRelease_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_Tr_PressRelease_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Pos_Fake_PressRelease_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neut_Fake_PressRelease_BlockChain'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']
news_clean_encoded['Neg_Fake_PressRelease_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_BlockChain']

# 'сategory_Cloud'
########################################################################
news_clean_encoded['Pos_Tr_news_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_Tr_news_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_Tr_news_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Pos_Fake_news_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_Fake_news_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_Fake_news_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_Cloud']

news_clean_encoded['Pos_Tr_PressRelease_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_Tr_PressRelease_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_Tr_PressRelease_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Pos_Fake_PressRelease_Cloud'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neut_Fake_PressRelease_Cloud'] = news_clean_encoded['sentiment_neutral']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']
news_clean_encoded['Neg_Fake_PressRelease_Cloud'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_press release'] * news_clean_encoded['сategory_Cloud']


agg_func = {}
for col in news_clean_encoded.columns:
    if col != 'Ticker':
        agg_func[col] = 'sum'
    else:
        agg_func[col] = 'first'

news_final = pd.DataFrame()

for ticker in news_clean_encoded['Ticker'].unique():
    slice_df = news_clean_encoded[news_clean_encoded['Ticker'] == ticker]
    slice_df = slice_df.resample('D').agg(agg_func)
    news_final = pd.concat([news_final, slice_df])


Index(['Ticker', 'sentiment_negative', 'sentiment_neutral',
       'sentiment_positive', 'fakenews_LABEL_0', 'fakenews_LABEL_1',
       'type_news', 'type_press release', 'сategory_AI',
       'сategory_Application', 'сategory_BigData', 'сategory_BlockChain',
       'сategory_Cloud', 'сategory_Other'],
      dtype='object')


<ipython-input-8-b83fb4d1e56f>:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_clean_encoded['Neg_Tr_news_BlockChain'] = news_clean_encoded['sentiment_negative']  * news_clean_encoded['fakenews_LABEL_0']*  news_clean_encoded['type_news'] * news_clean_encoded['сategory_BlockChain']
<ipython-input-8-b83fb4d1e56f>:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_clean_encoded['Pos_Fake_news_BlockChain'] = news_clean_encoded['sentiment_positive']  * news_clean_encoded['fakenews_LABEL_1']*  news_clean_encoded['type_new

In [ ]:
news_final.index = pd.to_datetime(news_final.index, errors='coerce')
news_final = news_final.reset_index().set_index(['Ticker', 'Date'])
news_final

sentiment_negative  sentiment_neutral  sentiment_positive  \
Ticker Date                                                                    
AAPL   2019-01-03                   1                  2                   2   
       2019-01-04                   1                  1                   0   
NaN    2019-01-05                   0                  0                   0   
       2019-01-06                   0                  0                   0   
AAPL   2019-01-07                   1                  1                   1   
...                               ...                ...                 ...   
NaN    2025-03-30                   0                  0                   0   
       2025-03-31                   0                  0                   0   
       2025-04-01                   0                  0                   0   
       2025-04-02                   0                  0                   0   
DOCU   2025-04-03                   0                  1                   0   

                   fakenews_LABEL_0  fakenews_LABEL_1  type_news  \
Ticker Date                                                        
AAPL   2019-01-03                 0                 5          5   
       2019-01-04                 1                 1          2   
NaN    2019-01-05                 0                 0          0   
       2019-01-06                 0                 0          0   
AAPL   2019-01-07                 1                 2          3   
...                             ...               ...        ...   
NaN    2025-03-30                 0                 0          0   
       2025-03-31                 0                 0          0   
       2025-04-01                 0                 0          0   
       2025-04-02                 0                 0          0   
DOCU   2025-04-03                 0                 1          0   

                   type_press release  сategory_AI  сategory_Application  \
Ticker Date                                                                
AAPL   2019-01-03                   0            0                     0   
       2019-01-04                   0            0                     0   
NaN    2019-01-05                   0            0                     0   
       2019-01-06                   0            0                     0   
AAPL   2019-01-07                   0            0                     0   
...                               ...          ...                   ...   
NaN    2025-03-30                   0            0                     0   
       2025-03-31                   0            0                     0   
       2025-04-01                   0            0                     0   
       2025-04-02                   0            0                     0   
DOCU   2025-04-03                   1            0                     0   

                   сategory_BigData  ...  Neg_Tr_news_Cloud  \
Ticker Date                          ...                      
AAPL   2019-01-03                 0  ...                  0   
       2019-01-04                 0  ...                  0   
NaN    2019-01-05                 0  ...                  0   
       2019-01-06                 0  ...                  0   
AAPL   2019-01-07                 0  ...                  0   
...                             ...  ...                ...   
NaN    2025-03-30                 0  ...                  0   
       2025-03-31                 0  ...                  0   
       2025-04-01                 0  ...                  0   
       2025-04-02                 0  ...                  0   
DOCU   2025-04-03                 0  ...                  0   

                   Pos_Fake_news_Cloud  Neut_Fake_news_Cloud  \
Ticker Date                                                    
AAPL   2019-01-03                    0                     0   
       2019-01-04                    0                     0   
NaN    2019-01-05           

In [ ]:
news_final = news_final.reset_index()

### Объединение финансового и новостей

In [ ]:
model_df = pd.merge(fin_df, news_final, left_index=True, right_index=True, how = 'left')
model_df

Open       High        Low      Close     Volume  \
Ticker Date                                                                
AAPL   2019-01-02  36.944466  37.889009  36.787041  37.667183  148158800   
       2019-01-03  34.342195  34.757222  33.869926  33.915245  365248800   
       2019-01-04  34.473394  35.432248  34.299275  35.363075  234428400   
       2019-01-07  35.468025  35.499034  34.800166  35.284363  219111200   
       2019-01-08  35.673145  36.212204  35.425085  35.956985  164101200   
...                      ...        ...        ...        ...        ...   
DOCU   2024-12-24  95.330002  96.320000  94.591003  94.849998    1287900   
       2024-12-26  94.040001  95.870003  93.949997  95.769997    1518700   
       2024-12-27  95.300003  95.449997  92.169998  93.000000    1510700   
       2024-12-30  92.099998  93.000000  90.639999  91.699997    2442900   
       2024-12-31  91.500000  92.440002  89.649002  89.940002    1760000   

                   Dividends  Stock Splits  Diluted Average Shares  \
Ticker Date                                                          
AAPL   2019-01-02        0.0           0.0            1.686492e+10   
       2019-01-03        0.0           0.0            1.686492e+10   
       2019-01-04        0.0           0.0            1.686492e+10   
       2019-01-07        0.0           0.0            1.686492e+10   
       2019-01-08        0.0           0.0            1.686492e+10   
...                      ...           ...                     ...   
DOCU   2024-12-24        0.0           0.0            2.089500e+08   
       2024-12-26        0.0           0.0            2.089500e+08   
       2024-12-27        0.0           0.0            2.089500e+08   
       2024-12-30        0.0           0.0            2.089500e+08   
       2024-12-31        0.0           0.0            2.089500e+08   

                     Market_Cap    company_name  ... Neg_Tr_news_Cloud  \
Ticker Date                                      ...                     
AAPL   2019-01-02  6.352540e+11      Apple Inc.  ...               NaN   
       2019-01-03  5.719779e+11      Apple Inc.  ...               0.0   
       2019-01-04  5.963954e+11      Apple Inc.  ...               0.0   
       2019-01-07  5.950679e+11      Apple Inc.  ...               0.0   
       2019-01-08  6.064116e+11      Apple Inc.  ...               0.0   
...                         ...             ...  ...               ...   
DOCU   2024-12-24  1.981891e+10  DocuSign, Inc.  ...               0.0   
       2024-12-26  2.001114e+10  DocuSign, Inc.  ...               0.0   
       2024-12-27  1.943235e+10  DocuSign, Inc.  ...               0.0   
       2024-12-30  1.916071e+10  DocuSign, Inc.  ...               0.0   
       2024-12-31  1.879296e+10  DocuSign, Inc.  ...               0.0   

                  Pos_Fake_news_Cloud  Neut_Fake_news_Cloud  \
Ticker Date                                                   
AAPL   2019-01-02                 NaN                   NaN   
       2019-01-03                 0.0                   0.0   
       2019-01-04                 0.0                   0.0   
       2019-01-07                 0.0                   0.0   
       2019-01-08                 0.0                   0.0   
...                               ...                   ...   
DOCU   2024-12-24                 0.0                   0.0   
       2024-12-26                 0.0                   0.0   
       2024-12-27                 0.0                   0.0   
       2024-12-30                 0.0                   0.0   
       2024-12-31                 0.0                   0.0   

                   Neg_Fake_news_Cloud  Pos_Tr_PressRelease_Cloud  \
Ticker Date                                                         
AAPL   2019-01-02                  NaN                        NaN   
       2019-01-03                  0.0                        0.0   
       2019-01-04                  0.0                        0.0

### Обработка

In [ ]:
model_df = model_df.reset_index()
model_df = model_df[model_df['Date'].dt.year < 2025]
model_df

,Date,Ticker,Abnormal_Return,Market_Cap,company_name,industry,sector,SP500_returns,SP500_returns_yesterday,Returns,...,Neg_Tr_PressRelease,Neg_Fake_PressRelease,Pos_Fake_PressRelease,Neut_Fake_PressRelease,сategory_AI,сategory_Application,сategory_BigData,сategory_BlockChain,сategory_Cloud,Year
0,2019-01-02,AAPL,0.004609,6.352540e+11,Apple Inc.,Consumer Electronics,Technology,0.013351,-0.017669,0.019562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,2019-01-03,AAPL,0.007358,5.719779e+11,Apple Inc.,Consumer Electronics,Technology,-0.017669,0.023283,-0.012432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
2,2019-01-04,AAPL,-0.000270,5.963954e+11,Apple Inc.,Consumer Electronics,Technology,0.023283,0.005553,0.025808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
3,2019-01-07,AAPL,-0.011398,5.950679e+11,Apple Inc.,Consumer Electronics,Technology,0.005553,0.002453,-0.005178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
4,2019-01-08,AAPL,0.005209,6.064116e+11,Apple Inc.,Consumer Electronics,Technology,0.002453,0.001922,0.007957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102711,2024-12-24,DOCU,-0.017014,1.981891e+10,"DocuSign, Inc.",Software - Application,Technology,0.009259,0.002095,-0.005035,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102712,2024-12-26,DOCU,0.015686,2.001114e+10,"DocuSign, Inc.",Software - Application,Technology,0.002095,-0.005882,0.018396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102713,2024-12-27,DOCU,-0.016524,1.943235e+10,"DocuSign, Inc.",Software - Application,Technology,-0.005882,-0.002319,-0.024134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102714,2024-12-30,DOCU,-0.001343,1.916071e+10,"DocuSign, Inc.",Software - Application,Technology,-0.002319,-0.006438,-0.004343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024


In [ ]:
len(model_df['Ticker'].unique())

76

In [ ]:
model_df.isnull().sum()

,0
Ticker,0
Date,0
Open,0
High,0
Low,0
...,...
Neut_Tr_PressRelease_Cloud,53891
Neg_Tr_PressRelease_Cloud,53891
Pos_Fake_PressRelease_Cloud,53891
Neut_Fake_PressRelease_Cloud,53891


In [ ]:
model_df.columns[90:]

Index(['Neg_Fake_PressRelease', 'Pos_Tr_news_AI', 'Neut_Tr_news_AI',
       'Neg_Tr_news_AI', 'Pos_Fake_news_AI', 'Neut_Fake_news_AI',
       'Neg_Fake_news_AI', 'Pos_Tr_PressRelease_AI', 'Neut_Tr_PressRelease_AI',
       'Neg_Tr_PressRelease_AI', 'Pos_Fake_PressRelease_AI',
       'Neut_Fake_PressRelease_AI', 'Neg_Fake_PressRelease_AI',
       'Pos_Tr_news_Application', 'Neut_Tr_news_Application',
       'Neg_Tr_news_Application', 'Pos_Fake_news_Application',
       'Neut_Fake_news_Application', 'Neg_Fake_news_Application',
       'Pos_Tr_PressRelease_Application', 'Neut_Tr_PressRelease_Application',
       'Neg_Tr_PressRelease_Application', 'Pos_Fake_PressRelease_Application',
       'Neut_Fake_PressRelease_Application',
       'Neg_Fake_PressRelease_Application', 'Pos_Tr_news_BigData',
       'Neut_Tr_news_BigData', 'Neg_Tr_news_BigData', 'Pos_Fake_news_BigData',
       'Neut_Fake_news_BigData', 'Neg_Fake_news_BigData',
       'Pos_Tr_PressRelease_BigData', 'Neut_Tr_PressRelease_Big

In [ ]:
model_df = model_df.rename(columns={'Total Assets': 'Total_Assets', 'P/E': 'PE', 'type_press release': 'type_press_release'})

In [ ]:
model_df = model_df[['Date', 'Ticker',
                     'Abnormal_Return',
                     'Market_Cap',
                     'company_name', 'industry', 'sector',
                     'SP500_returns', 'SP500_returns_yesterday',
                     'Returns', 'Returns_binary', 'Returns_yesterday',
                     'Total_Assets', 'ROA', 'ROE', 'LEV', 'PE', 'PBV', 'EPS', 'Beta',
                     'sentiment_negative', 'sentiment_neutral', 'sentiment_positive',
                     'fakenews_LABEL_0', 'fakenews_LABEL_1',
                     'type_news', 'type_press_release',


'Pos_news', 'Neg_news', 'Neut_news', 'Pos_PressRelease', 'Neg_PressRelease', 'Neut_PressRelease',


'Pos_news_AI', 'Neg_news_AI', 'Neut_news_AI', 'Pos_PressRelease_AI', 'Neg_PressRelease_AI', 'Neut_PressRelease_AI',

'Pos_news_Application','Neg_news_Application','Neut_news_Application','Pos_PressRelease_Application','Neg_PressRelease_Application','Neut_PressRelease_Application',

'Pos_news_BigData', 'Neg_news_BigData','Neut_news_BigData', 'Pos_PressRelease_BigData', 'Neg_PressRelease_BigData', 'Neut_PressRelease_BigData',

'Pos_news_BlockChain', 'Neg_news_BlockChain', 'Neut_news_BlockChain', 'Pos_PressRelease_BlockChain', 'Neg_PressRelease_BlockChain','Neut_PressRelease_BlockChain',

'Pos_news_Cloud', 'Neg_news_Cloud', 'Neut_news_Cloud', 'Pos_PressRelease_Cloud', 'Neg_PressRelease_Cloud', 'Neut_PressRelease_Cloud',



'Pos_Tr_news_AI', 'Neut_Tr_news_AI', 'Neg_Tr_news_AI', 'Pos_Fake_news_AI', 'Neut_Fake_news_AI','Neg_Fake_news_AI',
'Pos_Tr_PressRelease_AI', 'Neut_Tr_PressRelease_AI','Neg_Tr_PressRelease_AI', 'Pos_Fake_PressRelease_AI','Neut_Fake_PressRelease_AI', 'Neg_Fake_PressRelease_AI',

'Pos_Tr_news_Application',         'Neut_Tr_news_Application',        'Neg_Tr_news_Application',         'Pos_Fake_news_Application',        'Neut_Fake_news_Application',        'Neg_Fake_news_Application',
'Pos_Tr_PressRelease_Application', 'Neut_Tr_PressRelease_Application','Neg_Tr_PressRelease_Application', 'Pos_Fake_PressRelease_Application','Neut_Fake_PressRelease_Application','Neg_Fake_PressRelease_Application',

'Pos_Tr_news_BigData',        'Neut_Tr_news_BigData',        'Neg_Tr_news_BigData',        'Pos_Fake_news_BigData',        'Neut_Fake_news_BigData',        'Neg_Fake_news_BigData',
'Pos_Tr_PressRelease_BigData','Neut_Tr_PressRelease_BigData','Neg_Tr_PressRelease_BigData','Pos_Fake_PressRelease_BigData','Neut_Fake_PressRelease_BigData','Neg_Fake_PressRelease_BigData',

'Pos_Tr_news_BlockChain',        'Neut_Tr_news_BlockChain',        'Neg_Tr_news_BlockChain',        'Pos_Fake_news_BlockChain',        'Neut_Fake_news_BlockChain',        'Neg_Fake_news_BlockChain',
'Pos_Tr_PressRelease_BlockChain','Neut_Tr_PressRelease_BlockChain','Neg_Tr_PressRelease_BlockChain','Pos_Fake_PressRelease_BlockChain','Neut_Fake_PressRelease_BlockChain','Neg_Fake_PressRelease_BlockChain',

'Pos_Tr_news_Cloud',        'Neut_Tr_news_Cloud',        'Neg_Tr_news_Cloud',        'Pos_Fake_news_Cloud',        'Neut_Fake_news_Cloud',        'Neg_Fake_news_Cloud',
'Pos_Tr_PressRelease_Cloud','Neut_Tr_PressRelease_Cloud','Neg_Tr_PressRelease_Cloud','Pos_Fake_PressRelease_Cloud','Neut_Fake_PressRelease_Cloud','Neg_Fake_PressRelease_Cloud',



'Pos_Tr_news', 'Neut_Tr_news', 'Neg_Tr_news', 'Pos_Fake_news', 'Neut_Fake_news', 'Neg_Fake_news',

'Pos_Tr_PressRelease', 'Neut_Tr_PressRelease', 'Neg_Tr_PressRelease','Neg_Fake_PressRelease', 'Pos_Fake_PressRelease', 'Neut_Fake_PressRelease',

'сategory_AI', 'сategory_Application', 'сategory_BigData', 'сategory_BlockChain','сategory_Cloud', 'Year']]

In [ ]:
model_df

,Date,Ticker,Abnormal_Return,Market_Cap,company_name,industry,sector,SP500_returns,SP500_returns_yesterday,Returns,...,Neg_Tr_PressRelease,Neg_Fake_PressRelease,Pos_Fake_PressRelease,Neut_Fake_PressRelease,сategory_AI,сategory_Application,сategory_BigData,сategory_BlockChain,сategory_Cloud,Year
0,2019-01-02,AAPL,0.004609,6.352540e+11,Apple Inc.,Consumer Electronics,Technology,0.013351,-0.017669,0.019562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,2019-01-03,AAPL,0.007358,5.719779e+11,Apple Inc.,Consumer Electronics,Technology,-0.017669,0.023283,-0.012432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
2,2019-01-04,AAPL,-0.000270,5.963954e+11,Apple Inc.,Consumer Electronics,Technology,0.023283,0.005553,0.025808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
3,2019-01-07,AAPL,-0.011398,5.950679e+11,Apple Inc.,Consumer Electronics,Technology,0.005553,0.002453,-0.005178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
4,2019-01-08,AAPL,0.005209,6.064116e+11,Apple Inc.,Consumer Electronics,Technology,0.002453,0.001922,0.007957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102711,2024-12-24,DOCU,-0.017014,1.981891e+10,"DocuSign, Inc.",Software - Application,Technology,0.009259,0.002095,-0.005035,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102712,2024-12-26,DOCU,0.015686,2.001114e+10,"DocuSign, Inc.",Software - Application,Technology,0.002095,-0.005882,0.018396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102713,2024-12-27,DOCU,-0.016524,1.943235e+10,"DocuSign, Inc.",Software - Application,Technology,-0.005882,-0.002319,-0.024134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024
102714,2024-12-30,DOCU,-0.001343,1.916071e+10,"DocuSign, Inc.",Software - Application,Technology,-0.002319,-0.006438,-0.004343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024


## Модель

## my

In [ ]:
df = model_df.set_index(['Ticker', 'Date'])
df

Abnormal_Return    Market_Cap    company_name  \
Ticker Date                                                        
AAPL   2019-01-02         0.004609  6.352540e+11      Apple Inc.   
       2019-01-03         0.007358  5.719779e+11      Apple Inc.   
       2019-01-04        -0.000270  5.963954e+11      Apple Inc.   
       2019-01-07        -0.011398  5.950679e+11      Apple Inc.   
       2019-01-08         0.005209  6.064116e+11      Apple Inc.   
...                            ...           ...             ...   
DOCU   2024-12-24        -0.017014  1.981891e+10  DocuSign, Inc.   
       2024-12-26         0.015686  2.001114e+10  DocuSign, Inc.   
       2024-12-27        -0.016524  1.943235e+10  DocuSign, Inc.   
       2024-12-30        -0.001343  1.916071e+10  DocuSign, Inc.   
       2024-12-31        -0.008720  1.879296e+10  DocuSign, Inc.   

                                 industry      sector  SP500_returns  \
Ticker Date                                                            
AAPL   2019-01-02    Consumer Electronics  Technology       0.013351   
       2019-01-03    Consumer Electronics  Technology      -0.017669   
       2019-01-04    Consumer Electronics  Technology       0.023283   
       2019-01-07    Consumer Electronics  Technology       0.005553   
       2019-01-08    Consumer Electronics  Technology       0.002453   
...                                   ...         ...            ...   
DOCU   2024-12-24  Software - Application  Technology       0.009259   
       2024-12-26  Software - Application  Technology       0.002095   
       2024-12-27  Software - Application  Technology      -0.005882   
       2024-12-30  Software - Application  Technology      -0.002319   
       2024-12-31  Software - Application  Technology      -0.006438   

                   SP500_returns_yesterday   Returns  Returns_binary  \
Ticker Date                                                            
AAPL   2019-01-02                -0.017669  0.019562               1   
       2019-01-03                 0.023283 -0.012432               0   
       2019-01-04                 0.005553  0.025808               1   
       2019-01-07                 0.002453 -0.005178               0   
       2019-01-08                 0.001922  0.007957               1   
...                                    ...       ...             ...   
DOCU   2024-12-24                 0.002095 -0.005035               0   
       2024-12-26                -0.005882  0.018396               1   
       2024-12-27                -0.002319 -0.024134               0   
       2024-12-30                -0.006438 -0.004343               0   
       2024-12-31                 0.000105 -0.017049               0   

                   Returns_yesterday  ...  Neg_Tr_PressRelease  \
Ticker Date                           ...                        
AAPL   2019-01-02          -0.012432  ...                  NaN   
       2019-01-03           0.025808  ...                  0.0   
       2019-01-04          -0.005178  ...                  0.0   
       2019-01-07           0.007957  ...                  0.0   
       2019-01-08           0.013352  ...                  0.0   
...                              ...  ...                  ...   
DOCU   2024-12-24           0.018396  ...                  0.0   
       2024-12-26          -0.024134  ...                  0.0   
       2024-12-27          -0.004343  ...                  0.0   
       2024-12-30          -0.017049  ...                  0.0   
       2024-12-31           0.001223  ...                  0.0   

                   Neg_Fake_PressRelease  Pos_Fake_PressRelease  \
Ticker Date                                                       
AAPL   2019-01-02                    NaN                    NaN   
       2019-01-03                    0.0                    0.0   
       2019-01-04                    0.0                    0.0   
       2019-01-07                    0.0                    0.0   

## Model V2


### dataset

In [ ]:
!pip install linearmodels -q
from linearmodels.panel import PanelOLS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'stargazer'

In [ ]:
##

#df = pd.read_excel('/content/final_data.xlsx')
# Список столбцов, которые нужно логарифмировать
cols_to_log = ['Market_Cap', 'Total_Assets'] #, 'ROA', 'ROE', 'LEV', 'PE', 'PBV', 'EPS', 'Beta']
for col in cols_to_log:
    df[f'{col}'] = np.log(df[col] + 1)


# Выбросы на 10%
df_winsorized = df.copy()
numeric_cols = ['Market_Cap', 'Total_Assets', 'ROA', 'ROE', 'LEV', 'PE', 'PBV', 'EPS']

for col in numeric_cols:
    lower = df[col].quantile(0.10)
    upper = df[col].quantile(0.90)
    df_winsorized[col] = np.clip(df[col], lower, upper)

df = df_winsorized.copy()

#df = df.set_index(['Ticker', 'Date'])
df

Abnormal_Return  Market_Cap    company_name  \
Ticker Date                                                      
AAPL   2019-01-02         0.004609   26.139186      Apple Inc.   
       2019-01-03         0.007358   26.139186      Apple Inc.   
       2019-01-04        -0.000270   26.139186      Apple Inc.   
       2019-01-07        -0.011398   26.139186      Apple Inc.   
       2019-01-08         0.005209   26.139186      Apple Inc.   
...                            ...         ...             ...   
DOCU   2024-12-24        -0.017014   23.709902  DocuSign, Inc.   
       2024-12-26         0.015686   23.719555  DocuSign, Inc.   
       2024-12-27        -0.016524   23.690205  DocuSign, Inc.   
       2024-12-30        -0.001343   23.676128  DocuSign, Inc.   
       2024-12-31        -0.008720   23.656748  DocuSign, Inc.   

                                 industry      sector  SP500_returns  \
Ticker Date                                                            
AAPL   2019-01-02    Consumer Electronics  Technology       0.013351   
       2019-01-03    Consumer Electronics  Technology      -0.017669   
       2019-01-04    Consumer Electronics  Technology       0.023283   
       2019-01-07    Consumer Electronics  Technology       0.005553   
       2019-01-08    Consumer Electronics  Technology       0.002453   
...                                   ...         ...            ...   
DOCU   2024-12-24  Software - Application  Technology       0.009259   
       2024-12-26  Software - Application  Technology       0.002095   
       2024-12-27  Software - Application  Technology      -0.005882   
       2024-12-30  Software - Application  Technology      -0.002319   
       2024-12-31  Software - Application  Technology      -0.006438   

                   SP500_returns_yesterday   Returns  Returns_binary  \
Ticker Date                                                            
AAPL   2019-01-02                -0.017669  0.019562               1   
       2019-01-03                 0.023283 -0.012432               0   
       2019-01-04                 0.005553  0.025808               1   
       2019-01-07                 0.002453 -0.005178               0   
       2019-01-08                 0.001922  0.007957               1   
...                                    ...       ...             ...   
DOCU   2024-12-24                 0.002095 -0.005035               0   
       2024-12-26                -0.005882  0.018396               1   
       2024-12-27                -0.002319 -0.024134               0   
       2024-12-30                -0.006438 -0.004343               0   
       2024-12-31                 0.000105 -0.017049               0   

                   Returns_yesterday  ...  Neg_Tr_PressRelease  \
Ticker Date                           ...                        
AAPL   2019-01-02          -0.012432  ...                  NaN   
       2019-01-03           0.025808  ...                  0.0   
       2019-01-04          -0.005178  ...                  0.0   
       2019-01-07           0.007957  ...                  0.0   
       2019-01-08           0.013352  ...                  0.0   
...                              ...  ...                  ...   
DOCU   2024-12-24           0.018396  ...                  0.0   
       2024-12-26          -0.024134  ...                  0.0   
       2024-12-27          -0.004343  ...                  0.0   
       2024-12-30          -0.017049  ...                  0.0   
       2024-12-31           0.001223  ...                  0.0   

                   Neg_Fake_PressRelease  Pos_Fake_PressRelease  \
Ticker Date                                                       
AAPL   2019-01-02                    NaN                    NaN   
       2019-01-03                    0.0                    0.0   
       2019-01-04                    0.0                    0.0   
       2019-01-07                    0.0                    0.0   
       2019-01-08        

In [ ]:
## RUN TO GET SMOOTHING

smooth_vars = ['sentiment_negative', 'sentiment_neutral', 'sentiment_positive',
               'fakenews_LABEL_0', 'fakenews_LABEL_1',

               'Pos_Tr_news', 'Neut_Tr_news', 'Neg_Tr_news', 'Pos_Fake_news', 'Neut_Fake_news', 'Neg_Fake_news',
               'Pos_Tr_PressRelease', 'Neut_Tr_PressRelease', 'Neg_Tr_PressRelease',
               'Neg_Fake_PressRelease', 'Pos_Fake_PressRelease','Neut_Fake_PressRelease',

               'сategory_AI', 'сategory_Application', 'сategory_BigData', 'сategory_BlockChain', 'сategory_Cloud',

       'Pos_news', 'Neg_news', 'Neut_news',
       'Pos_PressRelease', 'Neg_PressRelease', 'Neut_PressRelease',

       'Pos_news_AI', 'Neg_news_AI', 'Neut_news_AI', 'Pos_PressRelease_AI',
       'Neg_PressRelease_AI', 'Neut_PressRelease_AI',

        'Pos_news_Application',
       'Neg_news_Application', 'Neut_news_Application',
       'Pos_PressRelease_Application', 'Neg_PressRelease_Application',
       'Neut_PressRelease_Application', 'Pos_news_BigData', 'Neg_news_BigData',
       'Neut_news_BigData', 'Pos_PressRelease_BigData',
       'Neg_PressRelease_BigData', 'Neut_PressRelease_BigData',
       'Pos_news_BlockChain', 'Neg_news_BlockChain', 'Neut_news_BlockChain',
       'Pos_PressRelease_BlockChain', 'Neg_PressRelease_BlockChain',
       'Neut_PressRelease_BlockChain', 'Pos_news_Cloud', 'Neg_news_Cloud',
       'Neut_news_Cloud', 'Pos_PressRelease_Cloud', 'Neg_PressRelease_Cloud',
       'Neut_PressRelease_Cloud',

'Pos_Tr_news_AI', 'Neut_Tr_news_AI', 'Neg_Tr_news_AI', 'Pos_Fake_news_AI', 'Neut_Fake_news_AI','Neg_Fake_news_AI',
'Pos_Tr_PressRelease_AI', 'Neut_Tr_PressRelease_AI','Neg_Tr_PressRelease_AI', 'Pos_Fake_PressRelease_AI','Neut_Fake_PressRelease_AI', 'Neg_Fake_PressRelease_AI',

'Pos_Tr_news_Application',         'Neut_Tr_news_Application',        'Neg_Tr_news_Application',         'Pos_Fake_news_Application',        'Neut_Fake_news_Application',        'Neg_Fake_news_Application',
'Pos_Tr_PressRelease_Application', 'Neut_Tr_PressRelease_Application','Neg_Tr_PressRelease_Application', 'Pos_Fake_PressRelease_Application','Neut_Fake_PressRelease_Application','Neg_Fake_PressRelease_Application',

'Pos_Tr_news_BigData',        'Neut_Tr_news_BigData',        'Neg_Tr_news_BigData',        'Pos_Fake_news_BigData',        'Neut_Fake_news_BigData',        'Neg_Fake_news_BigData',
'Pos_Tr_PressRelease_BigData','Neut_Tr_PressRelease_BigData','Neg_Tr_PressRelease_BigData','Pos_Fake_PressRelease_BigData','Neut_Fake_PressRelease_BigData','Neg_Fake_PressRelease_BigData',

'Pos_Tr_news_BlockChain',        'Neut_Tr_news_BlockChain',        'Neg_Tr_news_BlockChain',        'Pos_Fake_news_BlockChain',        'Neut_Fake_news_BlockChain',        'Neg_Fake_news_BlockChain',
'Pos_Tr_PressRelease_BlockChain','Neut_Tr_PressRelease_BlockChain','Neg_Tr_PressRelease_BlockChain','Pos_Fake_PressRelease_BlockChain','Neut_Fake_PressRelease_BlockChain','Neg_Fake_PressRelease_BlockChain',

'Pos_Tr_news_Cloud',        'Neut_Tr_news_Cloud',        'Neg_Tr_news_Cloud',        'Pos_Fake_news_Cloud',        'Neut_Fake_news_Cloud',        'Neg_Fake_news_Cloud',
'Pos_Tr_PressRelease_Cloud','Neut_Tr_PressRelease_Cloud','Neg_Tr_PressRelease_Cloud','Pos_Fake_PressRelease_Cloud','Neut_Fake_PressRelease_Cloud','Neg_Fake_PressRelease_Cloud'


       ]

window = 30
#for col in smooth_vars:
#        df[col] = (df.groupby(level=0)[col].transform(lambda s: s.rolling(window, min_periods=1).mean()))

In [ ]:
df

Abnormal_Return  Market_Cap    company_name  \
Ticker Date                                                      
AAPL   2019-01-02         0.004609   26.139186      Apple Inc.   
       2019-01-03         0.007358   26.139186      Apple Inc.   
       2019-01-04        -0.000270   26.139186      Apple Inc.   
       2019-01-07        -0.011398   26.139186      Apple Inc.   
       2019-01-08         0.005209   26.139186      Apple Inc.   
...                            ...         ...             ...   
DOCU   2024-12-24        -0.017014   23.709902  DocuSign, Inc.   
       2024-12-26         0.015686   23.719555  DocuSign, Inc.   
       2024-12-27        -0.016524   23.690205  DocuSign, Inc.   
       2024-12-30        -0.001343   23.676128  DocuSign, Inc.   
       2024-12-31        -0.008720   23.656748  DocuSign, Inc.   

                                 industry      sector  SP500_returns  \
Ticker Date                                                            
AAPL   2019-01-02    Consumer Electronics  Technology       0.013351   
       2019-01-03    Consumer Electronics  Technology      -0.017669   
       2019-01-04    Consumer Electronics  Technology       0.023283   
       2019-01-07    Consumer Electronics  Technology       0.005553   
       2019-01-08    Consumer Electronics  Technology       0.002453   
...                                   ...         ...            ...   
DOCU   2024-12-24  Software - Application  Technology       0.009259   
       2024-12-26  Software - Application  Technology       0.002095   
       2024-12-27  Software - Application  Technology      -0.005882   
       2024-12-30  Software - Application  Technology      -0.002319   
       2024-12-31  Software - Application  Technology      -0.006438   

                   SP500_returns_yesterday   Returns  Returns_binary  \
Ticker Date                                                            
AAPL   2019-01-02                -0.017669  0.019562               1   
       2019-01-03                 0.023283 -0.012432               0   
       2019-01-04                 0.005553  0.025808               1   
       2019-01-07                 0.002453 -0.005178               0   
       2019-01-08                 0.001922  0.007957               1   
...                                    ...       ...             ...   
DOCU   2024-12-24                 0.002095 -0.005035               0   
       2024-12-26                -0.005882  0.018396               1   
       2024-12-27                -0.002319 -0.024134               0   
       2024-12-30                -0.006438 -0.004343               0   
       2024-12-31                 0.000105 -0.017049               0   

                   Returns_yesterday  ...  Neg_Tr_PressRelease  \
Ticker Date                           ...                        
AAPL   2019-01-02          -0.012432  ...                  NaN   
       2019-01-03           0.025808  ...                  0.0   
       2019-01-04          -0.005178  ...                  0.0   
       2019-01-07           0.007957  ...                  0.0   
       2019-01-08           0.013352  ...                  0.0   
...                              ...  ...                  ...   
DOCU   2024-12-24           0.018396  ...                  0.0   
       2024-12-26          -0.024134  ...                  0.0   
       2024-12-27          -0.004343  ...                  0.0   
       2024-12-30          -0.017049  ...                  0.0   
       2024-12-31           0.001223  ...                  0.0   

                   Neg_Fake_PressRelease  Pos_Fake_PressRelease  \
Ticker Date                                                       
AAPL   2019-01-02                    NaN                    NaN   
       2019-01-03                    0.0                    0.0   
       2019-01-04                    0.0                    0.0   
       2019-01-07                    0.0                    0.0   
       2019-01-08        

### Models

In [ ]:
#df = df.reset_index()
#news_final = news_final.reset_index()
#news_final.to_excel("news_final.xlsx", index=False)

In [ ]:
#from google.colab import files
#files.download('last_final.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_excel('last_final.xlsx')

BadZipFile: File is not a zip file

In [ ]:
df = df.set_index(['Ticker', 'Date'])
df

Abnormal_Return  Market_Cap    company_name  \
Ticker Date                                                      
AAPL   2019-01-02         0.004609   26.139186      Apple Inc.   
       2019-01-03         0.007358   26.139186      Apple Inc.   
       2019-01-04        -0.000270   26.139186      Apple Inc.   
       2019-01-07        -0.011398   26.139186      Apple Inc.   
       2019-01-08         0.005209   26.139186      Apple Inc.   
...                            ...         ...             ...   
DOCU   2024-12-24        -0.017014   23.709902  DocuSign, Inc.   
       2024-12-26         0.015686   23.719555  DocuSign, Inc.   
       2024-12-27        -0.016524   23.690205  DocuSign, Inc.   
       2024-12-30        -0.001343   23.676128  DocuSign, Inc.   
       2024-12-31        -0.008720   23.656748  DocuSign, Inc.   

                                 industry      sector  SP500_returns  \
Ticker Date                                                            
AAPL   2019-01-02    Consumer Electronics  Technology       0.013351   
       2019-01-03    Consumer Electronics  Technology      -0.017669   
       2019-01-04    Consumer Electronics  Technology       0.023283   
       2019-01-07    Consumer Electronics  Technology       0.005553   
       2019-01-08    Consumer Electronics  Technology       0.002453   
...                                   ...         ...            ...   
DOCU   2024-12-24  Software - Application  Technology       0.009259   
       2024-12-26  Software - Application  Technology       0.002095   
       2024-12-27  Software - Application  Technology      -0.005882   
       2024-12-30  Software - Application  Technology      -0.002319   
       2024-12-31  Software - Application  Technology      -0.006438   

                   SP500_returns_yesterday   Returns  Returns_binary  \
Ticker Date                                                            
AAPL   2019-01-02                -0.017669  0.019562               1   
       2019-01-03                 0.023283 -0.012432               0   
       2019-01-04                 0.005553  0.025808               1   
       2019-01-07                 0.002453 -0.005178               0   
       2019-01-08                 0.001922  0.007957               1   
...                                    ...       ...             ...   
DOCU   2024-12-24                 0.002095 -0.005035               0   
       2024-12-26                -0.005882  0.018396               1   
       2024-12-27                -0.002319 -0.024134               0   
       2024-12-30                -0.006438 -0.004343               0   
       2024-12-31                 0.000105 -0.017049               0   

                   Returns_yesterday  ...  Neg_Tr_PressRelease  \
Ticker Date                           ...                        
AAPL   2019-01-02          -0.012432  ...                  NaN   
       2019-01-03           0.025808  ...                  0.0   
       2019-01-04          -0.005178  ...                  0.0   
       2019-01-07           0.007957  ...                  0.0   
       2019-01-08           0.013352  ...                  0.0   
...                              ...  ...                  ...   
DOCU   2024-12-24           0.018396  ...                  0.0   
       2024-12-26          -0.024134  ...                  0.0   
       2024-12-27          -0.004343  ...                  0.0   
       2024-12-30          -0.017049  ...                  0.0   
       2024-12-31           0.001223  ...                  0.0   

                   Neg_Fake_PressRelease  Pos_Fake_PressRelease  \
Ticker Date                                                       
AAPL   2019-01-02                    NaN                    NaN   
       2019-01-03                    0.0                    0.0   
       2019-01-04                    0.0                    0.0   
       2019-01-07                    0.0                    0.0   
       2019-01-08        

**Гипотеза 1 про тональность - без категорий**

In [ ]:
##mod = PanelOLS.from_formula('Abnormal_Return ~ Market_Cap + SP500_returns + Total_Assets + ROA + ROE + Beta + LEV + PE + PBV + EPS + constant + Year', df)
controls = ['Market_Cap', 'Total_Assets',  'ROE', 'LEV', 'PE', 'PBV', 'EPS']
X_vars = controls
mod = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res = mod.fit(cov_type="clustered", cluster_entity=True)



##mod = PanelOLS.from_formula('Abnormal_Return ~ Market_Cap + SP500_returns + Total_Assets + ROA + ROE + Beta + LEV + PE + PBV + EPS + constant + Year', df)
X_vars = controls +  ['sentiment_negative']
mod1 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_neg = mod1.fit(cov_type="clustered", cluster_entity=True)


X_vars = controls +  ['sentiment_positive']
mod2 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)


res_pos = mod2.fit(cov_type="clustered", cluster_entity=True)


X_vars = X_vars = controls +  ['sentiment_negative', 'sentiment_positive']
mod3 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_full = mod3.fit(cov_type="clustered", cluster_entity=True)

X_vars = X_vars = controls +  ['Pos_news']
mod4 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)
res_4 = mod4.fit(cov_type="clustered", cluster_entity=True)


X_vars = X_vars = controls +  ['Neg_news']
mod5 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)
res_5 = mod5.fit(cov_type="clustered", cluster_entity=True)

X_vars = X_vars = controls +  ['Pos_PressRelease']
mod6 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_6 = mod6.fit(cov_type="clustered", cluster_entity=True)

X_vars = X_vars = controls +  ['Neg_PressRelease']
mod7 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_7 = mod7.fit(cov_type="clustered", cluster_entity=True)

X_vars = X_vars = controls +  ['Pos_news', 'Neg_news']
mod8 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_8 = mod8.fit(cov_type="clustered", cluster_entity=True)

X_vars = X_vars = controls +  ['Pos_PressRelease', 'Neg_PressRelease']
mod9 = PanelOLS(df['Abnormal_Return'],
               df[X_vars],
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)

res_9 = mod9.fit(cov_type="clustered", cluster_entity=True)

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.1

In [ ]:
def add_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

# Модели и их названия
models = [
    (res, "Model 1 (Controls)"),
    (res_neg, "Model 2 (+ sentiment_negative)"),
    (res_pos, "Model 3 (+ sentiment_positive)"),
    (res_full, "Model 4 (+ sentiment_negative & positive)"),
    (res_4, "Model 5 (+ Pos_news)"),
    (res_5, "Model 6 (+ Neg_news)"),
    (res_6, "Model 7 (+ Pos_PressRelease)"),
    (res_7, "Model 8 (+ Neg_PressRelease)"),
    (res_8, "Model 9 (+ Pos & Neg news)"),
    (res_9, "Model 10 (+ Pos & Neg PressRelease)")]

# Все переменные, которые могут быть в моделях
all_vars = [
    'Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS',
    'sentiment_negative', 'sentiment_positive',
    'Pos_news', 'Neg_news', 'Pos_PressRelease', 'Neg_PressRelease']


table_rows = []

for res_i, label in models:
    row = {'Model': label}

    for var in all_vars:
        if var in res_i.params.index:
            coef = res_i.params[var]
            se = res_i.std_errors[var]
            pval = res_i.pvalues[var]
            row[f"Coef_{var}"] = f"{coef:.4f}{add_stars(pval)}"
            row[f"SE_{var}"] = f"{se:.4f}"
        else:
            row[f"Coef_{var}"] = ''
            row[f"SE_{var}"] = ''

    row['Observations'] = res_i.nobs
    row['R2'] = f"{res_i.rsquared:.4f}"
    row['Residual Std. Error'] = f"{res_i.resids.std():.4f}"
    if res_i.f_statistic is not None:
        row['F-statistic'] = f"{res_i.f_statistic.stat:.2f}{add_stars(res_i.f_statistic.pval)}"
    else:
        row['F-statistic'] = ''

    table_rows.append(row)

results_df = pd.DataFrame(table_rows)
results_df.T.to_excel("regression_sent1.xlsx")
results_df.T

,0,1,2,3,4,5,6,7,8,9
Model,Model 1 (Controls),Model 2 (+ sentiment_negative),Model 3 (+ sentiment_positive),Model 4 (+ sentiment_negative & positive),Model 5 (+ Pos_news),Model 6 (+ Neg_news),Model 7 (+ Pos_PressRelease),Model 8 (+ Neg_PressRelease),Model 9 (+ Pos & Neg news),Model 10 (+ Pos & Neg PressRelease)
Coef_Market_Cap,0.0019***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***
SE_Market_Cap,0.0005,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006
Coef_Total_Assets,-0.0014**,-0.0019***,-0.0020***,-0.0020***,-0.0020***,-0.0019***,-0.0019***,-0.0019***,-0.0020***,-0.0019***
SE_Total_Assets,0.0006,0.0007,0.0006,0.0006,0.0006,0.0007,0.0007,0.0007,0.0006,0.0007
Coef_ROE,0.0000,-0.0002,-0.0001,-0.0001,-0.0001,-0.0002,-0.0002,-0.0002,-0.0001,-0.0002
SE_ROE,0.0004,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008
Coef_LEV,0.0002,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*
SE_LEV,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003
Coef_PE,-0.0000*,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


**Гипотеза 1 про тональность - с категориями**

In [ ]:
from linearmodels.panel import PanelOLS
import pandas as pd

controls = ['Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS']
individual_vars = [
    'Pos_news_AI', 'Neg_news_AI', 'Pos_PressRelease_AI', 'Neg_PressRelease_AI',
    'Pos_news_Application', 'Neg_news_Application', 'Pos_PressRelease_Application', 'Neg_PressRelease_Application',
    'Pos_news_BigData', 'Neg_news_BigData', 'Pos_PressRelease_BigData', 'Neg_PressRelease_BigData',
    'Pos_news_BlockChain', 'Neg_news_BlockChain', 'Pos_PressRelease_BlockChain', 'Neg_PressRelease_BlockChain',
    'Pos_news_Cloud', 'Neg_news_Cloud', 'Pos_PressRelease_Cloud', 'Neg_PressRelease_Cloud']

results = []

# Функция для запуска модели
def run_model(name, additional_var, df):
    X_vars = controls + [additional_var]

    # Убираем переменные с одной уникальной категорией
    X_vars = [v for v in X_vars if v in df.columns and df[v].nunique() > 1]

    if additional_var not in X_vars:
        print(f"⚠️ Пропущена переменная: {additional_var} (недостаточная вариация или отсутствует)")
        return name, None

    try:
        model = PanelOLS(df['Abnormal_Return'], df[X_vars], entity_effects=True, time_effects=True, drop_absorbed=True)
        res = model.fit(cov_type="clustered", cluster_entity=True)
        return name, res
    except Exception as e:
        print(f"❌ Ошибка в {name}: {e}")
        return name, None

# Запуск моделей
for var in individual_vars:
    model_name = f'Model ({var})'
    results.append(run_model(model_name, var, df))

print("\n✅ Построенные модели:")
for name, res in results:
    if res is not None:
        print(name)


/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.1

⚠️ Пропущена переменная: Neg_PressRelease_Cloud (недостаточная вариация или отсутствует)

✅ Построенные модели:
Model (Pos_news_AI)
Model (Neg_news_AI)
Model (Pos_PressRelease_AI)
Model (Neg_PressRelease_AI)
Model (Pos_news_Application)
Model (Neg_news_Application)
Model (Pos_PressRelease_Application)
Model (Neg_PressRelease_Application)
Model (Pos_news_BigData)
Model (Neg_news_BigData)
Model (Pos_PressRelease_BigData)
Model (Neg_PressRelease_BigData)
Model (Pos_news_BlockChain)
Model (Neg_news_BlockChain)
Model (Pos_PressRelease_BlockChain)
Model (Neg_PressRelease_BlockChain)
Model (Pos_news_Cloud)
Model (Neg_news_Cloud)
Model (Pos_PressRelease_Cloud)


In [ ]:
import pandas as pd

# Функция для добавления звёздочек к p-value
def add_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

all_vars = [
    'Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS',
    'Pos_news_AI', 'Neg_news_AI', 'Pos_PressRelease_AI', 'Neg_PressRelease_AI',
    'Pos_news_Application', 'Neg_news_Application', 'Pos_PressRelease_Application', 'Neg_PressRelease_Application',
    'Pos_news_BigData', 'Neg_news_BigData', 'Pos_PressRelease_BigData', 'Neg_PressRelease_BigData',
    'Pos_news_BlockChain', 'Neg_news_BlockChain', 'Pos_PressRelease_BlockChain', 'Neg_PressRelease_BlockChain',
    'Pos_news_Cloud', 'Neg_news_Cloud', 'Pos_PressRelease_Cloud', 'Neg_PressRelease_Cloud']

table_rows = []

for model_name, res in results:
    row = {'Model': model_name}

    if res is not None:
        for var in all_vars:
            if var in res.params.index:
                coef = res.params[var]
                se = res.std_errors[var]
                pval = res.pvalues[var]
                row[f"Coef_{var}"] = f"{coef:.4f}{add_stars(pval)}"
                row[f"SE_{var}"] = f"{se:.4f}"
            else:
                row[f"Coef_{var}"] = ''
                row[f"SE_{var}"] = ''

        # Метрики модели
        row['Observations'] = res.nobs
        row['R2'] = f"{res.rsquared:.4f}"
        row['Residual Std. Error'] = f"{res.resids.std():.4f}"
        row['F-statistic'] = f"{res.f_statistic.stat:.2f}{add_stars(res.f_statistic.pval)}" if res.f_statistic is not None else ''
    else:
        # Если модель не рассчитана
        for var in all_vars:
            row[f"Coef_{var}"] = ''
            row[f"SE_{var}"] = ''
        row['Observations'] = ''
        row['R2'] = ''
        row['Residual Std. Error'] = ''
        row['F-statistic'] = ''

    table_rows.append(row)

# Итоговый DataFrame
results_df = pd.DataFrame(table_rows)

# Сохраняем в Excel
results_df.T.to_excel("regression_sent2.xlsx")

# Просмотр (по желанию)
results_df.T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Model,Model (Pos_news_AI),Model (Neg_news_AI),Model (Pos_PressRelease_AI),Model (Neg_PressRelease_AI),Model (Pos_news_Application),Model (Neg_news_Application),Model (Pos_PressRelease_Application),Model (Neg_PressRelease_Application),Model (Pos_news_BigData),Model (Neg_news_BigData),Model (Pos_PressRelease_BigData),Model (Neg_PressRelease_BigData),Model (Pos_news_BlockChain),Model (Neg_news_BlockChain),Model (Pos_PressRelease_BlockChain),Model (Neg_PressRelease_BlockChain),Model (Pos_news_Cloud),Model (Neg_news_Cloud),Model (Pos_PressRelease_Cloud),Model (Neg_PressRelease_Cloud)
Coef_Market_Cap,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,
SE_Market_Cap,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,
Coef_Total_Assets,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0020***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,
SE_Total_Assets,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0006,0.0006,0.0007,0.0007,0.0007,0.0007,0.0007,
Coef_ROE,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,
SE_ROE,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,
Coef_LEV,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,
SE_LEV,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,
Coef_PE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,


**Гипотеза 2 про фейки - без категорий**

In [ ]:
from linearmodels.panel import PanelOLS
import pandas as pd

controls = ['Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS']

results = []

def run_model(name, additional_vars, df):
    X_vars = controls + additional_vars
    model = PanelOLS(df['Abnormal_Return'], df[X_vars], entity_effects=True, time_effects=True, drop_absorbed=True)
    res = model.fit(cov_type="clustered", cluster_entity=True)
    return name, res

# TRUE NEWS
results.append(run_model('Model 1 (Controls)', [] , df))
results.append(run_model('Model 2 (Neg_Tr_news)', ['Neg_Tr_news'], df))
results.append(run_model('Model 3 (Pos_Tr_news)', ['Pos_Tr_news'], df))
results.append(run_model('Model 4 (Neg + Pos Tr_news)', ['Neg_Tr_news', 'Pos_Tr_news'], df))

# FAKE NEWS
results.append(run_model('Model 5 (Neg_Fake_news)', ['Neg_Fake_news'], df))
results.append(run_model('Model 6 (Pos_Fake_news)', ['Pos_Fake_news'], df))
results.append(run_model('Model 7 (Neg + Pos Fake_news)', ['Neg_Fake_news', 'Pos_Fake_news'], df))
results.append(run_model('Model 8 (All News)', ['Neg_Tr_news', 'Pos_Tr_news', 'Neg_Fake_news', 'Pos_Fake_news'], df))

# TRUE PRESS RELEASE
results.append(run_model('Model 9 (Neg_Tr_PressRelease)', ['Neg_Tr_PressRelease'], df))
results.append(run_model('Model 10 (Pos_Tr_PressRelease)', ['Pos_Tr_PressRelease'], df))
results.append(run_model('Model 11 (Neg + Pos Tr_PressRelease)', ['Neg_Tr_PressRelease', 'Pos_Tr_PressRelease'], df))

# FAKE PRESS RELEASE
results.append(run_model('Model 12 (Neg_Fake_PressRelease)', ['Neg_Fake_PressRelease'], df))
results.append(run_model('Model 13 (Pos_Fake_PressRelease)', ['Pos_Fake_PressRelease'], df))
results.append(run_model('Model 14 (Neg + Pos Fake_PressRelease)', ['Neg_Fake_PressRelease', 'Pos_Fake_PressRelease'], df))
results.append(run_model('Model 15 (All PressRelease)', ['Neg_Tr_PressRelease', 'Pos_Tr_PressRelease', 'Neg_Fake_PressRelease', 'Pos_Fake_PressRelease'], df))


/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.1

In [ ]:
import pandas as pd

# Функция для добавления звёздочек к p-value
def add_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

# Все переменные, которые могут быть в моделях
all_vars = [
    'Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS',
    'Neg_Tr_news', 'Pos_Tr_news', 'Neg_Fake_news', 'Pos_Fake_news',
    'Neg_Tr_PressRelease', 'Pos_Tr_PressRelease',
    'Neg_Fake_PressRelease', 'Pos_Fake_PressRelease'
]

# Собираем таблицу результатов
table_rows = []

for model_name, res in results:
    row = {'Model': model_name}

    for var in all_vars:
        if var in res.params.index:
            coef = res.params[var]
            se = res.std_errors[var]
            pval = res.pvalues[var]
            row[f"Coef_{var}"] = f"{coef:.4f}{add_stars(pval)}"
            row[f"SE_{var}"] = f"{se:.4f}"
        else:
            row[f"Coef_{var}"] = ''
            row[f"SE_{var}"] = ''

    # Метрики модели
    row['Observations'] = res.nobs
    row['R2'] = f"{res.rsquared:.4f}"
    row['Residual Std. Error'] = f"{res.resids.std():.4f}"

    # F-статистика
    if res.f_statistic is not None:
        row['F-statistic'] = f"{res.f_statistic.stat:.2f}{add_stars(res.f_statistic.pval)}"
    else:
        row['F-statistic'] = ''

    table_rows.append(row)

# Итоговый DataFrame
results_df = pd.DataFrame(table_rows)

# Сохраняем результат (по строкам — модели, по столбцам — коэффициенты и метрики)
results_df.T.to_excel("regression_fake1.xlsx")

# Если хочешь — транспонируем и смотрим
results_df.T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Model,Model 1 (Controls),Model 2 (Neg_Tr_news),Model 3 (Pos_Tr_news),Model 4 (Neg + Pos Tr_news),Model 5 (Neg_Fake_news),Model 6 (Pos_Fake_news),Model 7 (Neg + Pos Fake_news),Model 8 (All News),Model 9 (Neg_Tr_PressRelease),Model 10 (Pos_Tr_PressRelease),Model 11 (Neg + Pos Tr_PressRelease),Model 12 (Neg_Fake_PressRelease),Model 13 (Pos_Fake_PressRelease),Model 14 (Neg + Pos Fake_PressRelease),Model 15 (All PressRelease)
Coef_Market_Cap,0.0019***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***
SE_Market_Cap,0.0005,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006
Coef_Total_Assets,-0.0014**,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0020***,-0.0020***,-0.0020***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***
SE_Total_Assets,0.0006,0.0006,0.0006,0.0006,0.0007,0.0006,0.0006,0.0006,0.0006,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007
Coef_ROE,0.0000,-0.0002,-0.0002,-0.0002,-0.0002,-0.0001,-0.0001,-0.0001,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002,-0.0002
SE_ROE,0.0004,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008
Coef_LEV,0.0002,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*,0.0006*
SE_LEV,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003
Coef_PE,-0.0000*,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


**Гипотеза 2 про фейковость - с категориями**

In [ ]:
from linearmodels.panel import PanelOLS
import pandas as pd

controls = ['Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS']

results = []

def run_model(name, additional_vars, df):
    X_vars = controls + additional_vars
    model = PanelOLS(df['Abnormal_Return'], df[X_vars], entity_effects=True, time_effects=True, drop_absorbed=True)
    res = model.fit(cov_type="clustered", cluster_entity=True)
    return name, res

# _AI
results.append(run_model('Model 0 (Controls)', [] , df))
results.append(run_model('Model_1_AI', ['Pos_Tr_news_AI'], df))
#results.append(run_model('Model_2_AI', ['Neut_Tr_news_AI'], df))
results.append(run_model('Model_3_AI', ['Neg_Tr_news_AI'], df))
results.append(run_model('Model_4_AI', ['Pos_Fake_news_AI'], df))
#results.append(run_model('Model_5_AI', ['Neut_Fake_news_AI'], df))
results.append(run_model('Model_6_AI', ['Neg_Fake_news_AI'], df))
results.append(run_model('Model_7_AI', ['Pos_Tr_PressRelease_AI'], df))
#results.append(run_model('Model_8_AI', ['Neut_Tr_PressRelease_AI'], df))
results.append(run_model('Model_9_AI', ['Neg_Tr_PressRelease_AI'], df))
results.append(run_model('Model_10_AI', ['Pos_Fake_PressRelease_AI'], df))
#results.append(run_model('Model_11_AI', ['Neut_Fake_PressRelease_AI'], df))
results.append(run_model('Model_12_AI', ['Neg_Fake_PressRelease_AI'], df))

# _Application
results.append(run_model('Model_Application_1', ['Pos_Tr_news_Application'], df))
#results.append(run_model('Model_Application_2', ['Neut_Tr_news_Application'], df))
results.append(run_model('Model_Application_3', ['Neg_Tr_news_Application'], df))
results.append(run_model('Model_Application_4', ['Pos_Fake_news_Application'], df))
#results.append(run_model('Model_Application_5', ['Neut_Fake_news_Application'], df))
results.append(run_model('Model_Application_6', ['Neg_Fake_news_Application'], df))
results.append(run_model('Model_Application_7', ['Pos_Tr_PressRelease_Application'], df))
#results.append(run_model('Model_Application_8', ['Neut_Tr_PressRelease_Application'], df))
results.append(run_model('Model_Application_9', ['Neg_Tr_PressRelease_Application'], df))
results.append(run_model('Model_Application_10', ['Pos_Fake_PressRelease_Application'], df))
#results.append(run_model('Model_Application_11', ['Neut_Fake_PressRelease_Application'], df))
results.append(run_model('Model_Application_12', ['Neg_Fake_PressRelease_Application'], df))

# _BigData
results.append(run_model('Model_BigData1', ['Pos_Tr_news_BigData'], df))
#results.append(run_model('Model_BigData2', ['Neut_Tr_news_BigData'], df))
results.append(run_model('Model_BigData3', ['Neg_Tr_news_BigData'], df))
results.append(run_model('Model_BigData4', ['Pos_Fake_news_BigData'], df))
#results.append(run_model('Model_BigData5', ['Neut_Fake_news_BigData'], df))
results.append(run_model('Model_BigData6', ['Neg_Fake_news_BigData'], df))
results.append(run_model('Model_BigData7', ['Pos_Tr_PressRelease_BigData'], df))
#results.append(run_model('Model_BigData8', ['Neut_Tr_PressRelease_BigData'], df))
results.append(run_model('Model_BigData9', ['Neg_Tr_PressRelease_BigData'], df))
results.append(run_model('Model_BigData10', ['Pos_Fake_PressRelease_BigData'], df))
#results.append(run_model('Model_BigData11', ['Neut_Fake_PressRelease_BigData'], df))
results.append(run_model('Model_BigData12', ['Neg_Fake_PressRelease_BigData'], df))

# _BlockChain
results.append(run_model('Model_BlockChain1', ['Pos_Tr_news_BlockChain'], df))
#results.append(run_model('Model_BlockChain2', ['Neut_Tr_news_BlockChain'], df))
results.append(run_model('Model_BlockChain3', ['Neg_Tr_news_BlockChain'], df))
results.append(run_model('Model_BlockChain4', ['Pos_Fake_news_BlockChain'], df))
#results.append(run_model('Model_BlockChain5', ['Neut_Fake_news_BlockChain'], df))
results.append(run_model('Model_BlockChain6', ['Neg_Fake_news_BlockChain'], df))
results.append(run_model('Model_BlockChain7', ['Pos_Tr_PressRelease_BlockChain'], df))
#results.append(run_model('Model_BlockChain8', ['Neut_Tr_PressRelease_BlockChain'], df))
results.append(run_model('Model_BlockChain9', ['Neg_Tr_PressRelease_BlockChain'], df))
results.append(run_model('Model_BlockChain10', ['Pos_Fake_PressRelease_BlockChain'], df))
#results.append(run_model('Model_BlockChain11', ['Neut_Fake_PressRelease_BlockChain'], df))
results.append(run_model('Model_BlockChain12', ['Neg_Fake_PressRelease_BlockChain'], df))

# _Cloud
results.append(run_model('Model_Cloud1', ['Pos_Tr_news_Cloud'], df))
#results.append(run_model('Model_Cloud2', ['Neut_Tr_news_Cloud'], df))
results.append(run_model('Model_Cloud3', ['Neg_Tr_news_Cloud'], df))
results.append(run_model('Model_Cloud4', ['Pos_Fake_news_Cloud'], df))
#results.append(run_model('Model_Cloud5', ['Neut_Fake_news_Cloud'], df))
results.append(run_model('Model_Cloud6', ['Neg_Fake_news_Cloud'], df))
results.append(run_model('Model_Cloud7', ['Pos_Tr_PressRelease_Cloud'], df))
#results.append(run_model('Model_Cloud8', ['Neut_Tr_PressRelease_Cloud'], df))
#results.append(run_model('Model_Cloud9', ['Neg_Tr_PressRelease_Cloud'], df))
results.append(run_model('Model_Cloud10', ['Pos_Fake_PressRelease_Cloud'], df))
#results.append(run_model('Model_Cloud11', ['Neut_Fake_PressRelease_Cloud'], df))
#results.append(run_model('Model_Cloud12', ['Neg_Fake_PressRelease_Cloud'], df))


/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.1

In [ ]:
import pandas as pd

# Функция для добавления звёздочек к p-value
def add_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

# Список контролей (их всегда добавляем)
control_vars = ['Market_Cap', 'Total_Assets', 'ROE', 'LEV', 'PE', 'PBV', 'EPS']

# Собираем таблицу результатов
table_rows = []

for model_name, res in results:
    row = {'Model': model_name}

    for var in res.params.index:
        coef = res.params[var]
        se = res.std_errors[var]
        pval = res.pvalues[var]
        row[f"Coef_{var}"] = f"{coef:.4f}{add_stars(pval)}"
        row[f"SE_{var}"] = f"{se:.4f}"

    # Метрики модели
    row['Observations'] = res.nobs
    row['R2'] = f"{res.rsquared:.4f}"
    row['Residual Std. Error'] = f"{res.resids.std():.4f}"

    # F-статистика
    if res.f_statistic is not None:
        row['F-statistic'] = f"{res.f_statistic.stat:.2f}{add_stars(res.f_statistic.pval)}"
    else:
        row['F-statistic'] = ''

    table_rows.append(row)

# Итоговый DataFrame
results_df = pd.DataFrame(table_rows)

# Сохраняем в Excel
results_df.T.to_excel("regression_fake2.xlsx")
results_df.T

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
Model,Model 0 (Controls),Model_1_AI,Model_3_AI,Model_4_AI,Model_6_AI,Model_7_AI,Model_9_AI,Model_10_AI,Model_12_AI,Model_Application_1,...,Model_BlockChain7,Model_BlockChain9,Model_BlockChain10,Model_BlockChain12,Model_Cloud1,Model_Cloud3,Model_Cloud4,Model_Cloud6,Model_Cloud7,Model_Cloud10
Coef_Market_Cap,0.0019***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,...,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***,0.0026***
SE_Market_Cap,0.0005,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,...,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006
Coef_Total_Assets,-0.0014**,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,...,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***,-0.0019***
SE_Total_Assets,0.0006,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,...,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE_Neg_Fake_news_Cloud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0015,NaN,NaN
Coef_Pos_Tr_PressRelease_Cloud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0019,NaN
SE_Pos_Tr_PressRelease_Cloud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0037,NaN
Coef_Pos_Fake_PressRelease_Cloud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0061**


In [ ]:
df[['Pos_Tr_news_AI', 'Neut_Tr_news_AI', 'Neg_Tr_news_AI', 'Pos_Fake_news_AI', 'Neut_Fake_news_AI','Neg_Fake_news_AI',
'Pos_Tr_PressRelease_AI', 'Neut_Tr_PressRelease_AI','Neg_Tr_PressRelease_AI', 'Pos_Fake_PressRelease_AI','Neut_Fake_PressRelease_AI', 'Neg_Fake_PressRelease_AI',

'Pos_Tr_news_Application',         'Neut_Tr_news_Application',        'Neg_Tr_news_Application',         'Pos_Fake_news_Application',        'Neut_Fake_news_Application',        'Neg_Fake_news_Application',
'Pos_Tr_PressRelease_Application', 'Neut_Tr_PressRelease_Application','Neg_Tr_PressRelease_Application', 'Pos_Fake_PressRelease_Application','Neut_Fake_PressRelease_Application','Neg_Fake_PressRelease_Application',

'Pos_Tr_news_BigData',        'Neut_Tr_news_BigData',        'Neg_Tr_news_BigData',        'Pos_Fake_news_BigData',        'Neut_Fake_news_BigData',        'Neg_Fake_news_BigData',
'Pos_Tr_PressRelease_BigData','Neut_Tr_PressRelease_BigData','Neg_Tr_PressRelease_BigData','Pos_Fake_PressRelease_BigData','Neut_Fake_PressRelease_BigData','Neg_Fake_PressRelease_BigData',

'Pos_Tr_news_BlockChain',        'Neut_Tr_news_BlockChain',        'Neg_Tr_news_BlockChain',        'Pos_Fake_news_BlockChain',        'Neut_Fake_news_BlockChain',        'Neg_Fake_news_BlockChain',
'Pos_Tr_PressRelease_BlockChain','Neut_Tr_PressRelease_BlockChain','Neg_Tr_PressRelease_BlockChain','Pos_Fake_PressRelease_BlockChain','Neut_Fake_PressRelease_BlockChain','Neg_Fake_PressRelease_BlockChain',

'Pos_Tr_news_Cloud',        'Neut_Tr_news_Cloud',        'Neg_Tr_news_Cloud',        'Pos_Fake_news_Cloud',        'Neut_Fake_news_Cloud',        'Neg_Fake_news_Cloud',
'Pos_Tr_PressRelease_Cloud','Neut_Tr_PressRelease_Cloud','Neg_Tr_PressRelease_Cloud','Pos_Fake_PressRelease_Cloud','Neut_Fake_PressRelease_Cloud',
    'Neg_Fake_PressRelease_Cloud']].describe()



,Pos_Tr_news_AI,Neut_Tr_news_AI,Neg_Tr_news_AI,Pos_Fake_news_AI,Neut_Fake_news_AI,Neg_Fake_news_AI,Pos_Tr_PressRelease_AI,Neut_Tr_PressRelease_AI,Neg_Tr_PressRelease_AI,Pos_Fake_PressRelease_AI,...,Neg_Tr_news_Cloud,Pos_Fake_news_Cloud,Neut_Fake_news_Cloud,Neg_Fake_news_Cloud,Pos_Tr_PressRelease_Cloud,Neut_Tr_PressRelease_Cloud,Neg_Tr_PressRelease_Cloud,Pos_Fake_PressRelease_Cloud,Neut_Fake_PressRelease_Cloud,Neg_Fake_PressRelease_Cloud
count,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,...,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.000000,17793.0,17793.000000,17793.000000,17793.0
mean,0.023605,0.118586,0.012589,0.038273,0.128815,0.024673,0.005002,0.005283,0.000506,0.006295,...,0.000674,0.006519,0.021469,0.003091,0.002810,0.002529,0.0,0.003204,0.003035,0.0
std,0.163923,0.373331,0.119290,0.232157,0.409720,0.175847,0.072125,0.078452,0.022485,0.080499,...,0.025962,0.089727,0.162150,0.061288,0.058003,0.051335,0.0,0.060358,0.058953,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
max,3.000000,7.000000,4.000000,7.000000,5.000000,8.000000,2.000000,3.000000,1.000000,2.000000,...,1.000000,5.000000,4.000000,3.000000,3.000000,2.000000,0.0,2.000000,2.000000,0.0
